Create Event With Verified Domain
============================


In [1]:
# Create an event for an asset given url to Archivist and user Token.
#
# The module contains four functions: main, create_asset, create_event and
# get_verified_domain.
#
# The main function would initialize an archivist connection using the url and
# the credentials, called "arch".

# create_asset will execute 'assets.create', which is a archivist connection function
# to create a new asset for the archivist through archivist connection. The main funciton then
# calls create_event and pass in "arch" and the created asset to create a new event for the asset.

# In both cases the verified domain name is displayed.

In [2]:
from json import dumps as json_dumps
from os import getenv
from warnings import filterwarnings

from archivist.archivist import Archivist
from archivist.logger import set_logger

filterwarnings("ignore", message="Unverified HTTPS request")

In [3]:
# URL, CLIENT, SECRET are environment variables that represent connection parameters.
#
# URL = represents the url to the RKVST application
# CLIENT = represents the client ID from an Application Registration
# SECRET = represents the client secret from an Application Registration
RKVST_URL = getenv("RKVST_URL")
APPREG_CLIENT = getenv("RKVST_APPREG_CLIENT")
APPREG_SECRET = getenv("RKVST_APPREG_SECRET")

In [4]:
"""
Main function of Asset and Event creation.

* Connect to RKVST with client ID and client secret
* Creates an Asset and two Events
* Prints response of Asset and Event creation
"""

# Optional call to set the logger level.  The argument can be either
# "INFO" or "DEBUG".  For more sophisticated logging control see our
# documentation.
set_logger("INFO")

# Initialize connection to RKVST
print("Connecting to RKVST")
print("URL", RKVST_URL)
arch = Archivist(RKVST_URL, (APPREG_CLIENT, APPREG_SECRET), max_time=300)

Connecting to RKVST
URL https://app.dev-paul-0.wild.jitsuin.io


In [5]:
def get_verified_domain(arch, entity):
    """Get the verified domain for the asset/event.

    Args:
        arch: archivist connection.
        entity: an asset or event

    Returns:
        verified_domain: name of the verified domain
                         for the asset or event.

    """

    tenancy = arch.tenancies.publicinfo(entity["tenant_identity"])
    return tenancy.get("verified_domain", "")

In [6]:
def create_event(arch, asset):
    """Create an event for the passed-in asset.

    Args:
        arch: archivist connection.
        asset: an asset created using aconn

    Returns:
        new_event: a new event for the asset.

    """

    # props can be defined for different behaviours and the attributes associated with
    # different behaviours are also different.
    props = {
        "operation": "Record",
        # This event is used to record evidence.
        "behaviour": "RecordEvidence",
        # Optional Client-claimed time at which the maintenance was performed
        "timestamp_declared": "2019-11-27T14:44:19Z",
        # Optional Client-claimed identity of person performing the operation
        "principal_declared": {
            "issuer": "idp.synsation.io/1234",
            "subject": "phil.b",
            "email": "phil.b@synsation.io",
        },
    }
    attrs = {
        # Required Details of the RecordEvidence request
        "arc_description": "Safety conformance approved for version 1.6.",
        # Required The evidence to be retained in the asset history
        "arc_evidence": "DVA Conformance Report attached",
        # Example Client can add any additional information in further attributes,
        # including free text or attachments
        "conformance_report": "blobs/e2a1d16c-03cd-45a1-8cd0-690831df1273",
    }

    return arch.events.create(asset["identity"], props=props, attrs=attrs, confirm=True)

    # alternatively if some work can be done whilst the event is confirmed then this call can be
    # replaced by a two-step alternative:

    # event = arch.events.create(asset["identity"], props=props, attrs=attrs, confirm=False)

    # ... do something else here
    # and then wait for confirmation

    # self.arch.events.wait_for_confirmation(event['identity'])

In [7]:
def create_asset(arch):
    """Create an asset using Archivist Connection.

    Args:
        arch: archivist connection.

    Returns:
        newasset: a new asset created.

    """
    attrs = {
        "arc_display_name": "display_name",  # Asset's display name in the user interface
        "arc_description": "display_description",  # Asset's description in the user interface
        "arc_display_type": "desplay_type",  # Arc_display_type is a free text field
        # allowing the creator of
        # an asset to specify the asset
        # type or class. Be careful when setting this:
        # assets are grouped by type and
        # sharing policies can be
        # configured to share assets based on
        # their arc_display_type.
        # So a mistake here can result in asset data being
        # under- or over-shared.
        "some_custom_attribute": "value"  # You can add any custom value as long as
        # it does not start with arc_
    }
    # The first argument is the attributes of the asset
    # The second argument is wait for confirmation:
    #   If @confirm@ is True then this function will not
    #   return until the asset is confirmed on the blockchain and ready
    #   to accept events (or an error occurs)
    #   After an asset is submitted to the blockchain (submitted),
    #   it will be in the "Pending" status.
    #   Once it is added to the blockchain, the status will be changed to "Confirmed"
    return arch.assets.create(attrs=attrs, confirm=True)

In [8]:
# Create a new asset
asset = create_asset(arch)
print("Asset", json_dumps(asset, sort_keys=True, indent=4))
print("Verified domain '", get_verified_domain(arch, asset), "'")

Refresh token


Asset {
    "at_time": "2023-01-10T16:22:02Z",
    "attributes": {
        "arc_description": "display_description",
        "arc_display_name": "display_name",
        "arc_display_type": "desplay_type",
        "some_custom_attribute": "value"
    },
    "behaviours": [
        "Attachments",
        "RecordEvidence",
        "Builtin",
        "AssetCreator"
    ],
    "chain_id": "99",
    "confirmation_status": "CONFIRMED",
    "identity": "assets/bee28a3d-88e8-4906-8c73-2aaa48243572",
    "owner": "0x3E06AcBf002E8F67bDe832b3f3E7d2aDA2BE3DC8",
    "proof_mechanism": "SIMPLE_HASH",
    "public": false,
    "storage_integrity": "TENANT_STORAGE",
    "tenant_identity": "tenant/f9040e8b-2afa-48f1-bb00-f055d55ade01",
    "tracked": "TRACKED"
}
Verified domain '  '


In [9]:
# Create a new event
event = create_event(arch, asset)
print("Event", json_dumps(event, sort_keys=True, indent=4))
print("Verified domain '", get_verified_domain(arch, event), "'")

Event {
    "asset_attributes": {},
    "asset_identity": "assets/bee28a3d-88e8-4906-8c73-2aaa48243572",
    "behaviour": "RecordEvidence",
    "block_number": 0,
    "confirmation_status": "CONFIRMED",
    "event_attributes": {
        "arc_description": "Safety conformance approved for version 1.6.",
        "arc_evidence": "DVA Conformance Report attached",
        "conformance_report": "blobs/e2a1d16c-03cd-45a1-8cd0-690831df1273"
    },
    "from": "0x3E06AcBf002E8F67bDe832b3f3E7d2aDA2BE3DC8",
    "identity": "assets/bee28a3d-88e8-4906-8c73-2aaa48243572/events/2a7aaf85-e927-4170-bbe2-60995696c333",
    "operation": "Record",
    "principal_accepted": {
        "display_name": "Application display name 05f19055-d2fd-4076-bd02-46a924cbe984",
        "email": "",
        "issuer": "https://app.dev-paul-0.wild.jitsuin.io/appidpv1",
        "subject": "bd3f85d2-9b92-4ef0-82c9-2bcb7b5d25c6"
    },
    "principal_declared": {
        "display_name": "",
        "email": "phil.b@synsation.

In [10]:
# Fetch the event
event = arch.events.read(event["identity"])
print("Event", json_dumps(event, sort_keys=True, indent=4))
print("Verified domain '", get_verified_domain(arch, event), "'")

Event {
    "asset_attributes": {},
    "asset_identity": "assets/bee28a3d-88e8-4906-8c73-2aaa48243572",
    "behaviour": "RecordEvidence",
    "block_number": 0,
    "confirmation_status": "CONFIRMED",
    "event_attributes": {
        "arc_description": "Safety conformance approved for version 1.6.",
        "arc_evidence": "DVA Conformance Report attached",
        "conformance_report": "blobs/e2a1d16c-03cd-45a1-8cd0-690831df1273"
    },
    "from": "0x3E06AcBf002E8F67bDe832b3f3E7d2aDA2BE3DC8",
    "identity": "assets/bee28a3d-88e8-4906-8c73-2aaa48243572/events/2a7aaf85-e927-4170-bbe2-60995696c333",
    "operation": "Record",
    "principal_accepted": {
        "display_name": "Application display name 05f19055-d2fd-4076-bd02-46a924cbe984",
        "email": "",
        "issuer": "https://app.dev-paul-0.wild.jitsuin.io/appidpv1",
        "subject": "bd3f85d2-9b92-4ef0-82c9-2bcb7b5d25c6"
    },
    "principal_declared": {
        "display_name": "",
        "email": "phil.b@synsation.